# LAPD Crime Stats 

## Dataset column descriptions:

source: https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8

<ul>
    <li>dr_no: Division of records number</li>
    <li>date_rptd: Date reported </li>
    <li>date_occ: Date Incident occurred</li>
    <li>time_occ: Time Incident occurred</li>
    <li>area: Police station (numbered 1 - 21)</li>
    <li>area_name: Name of area station</li>
    <li>rpt_dist_no: Reporting district number</li>
    <li>part_1_2: Part I or Part II level offense</li>
    <li>crm_cd: Crime committed</li>
    <li>Crm Cd Desc: Description of crime committed</li>
    <li>Mocodes: Modus Operandi code activity of associated suspect</li>
    <li>Vict Age: Age of victim</li>
    <li>Vict Sex: Sex of victim</li>
        <ul>
            <li>M: Male</li>
            <li>F: Female</li>
            <li>X: Unknown</li>
        </ul>
    <li>Vict Descent: Descent codes:</li>
        <ul>
            <li>A: Other Asian</li>
            <li>B: Black</li>
            <li>C: Chinese</li>
            <li>D: Cambodian</li>
            <li>F: Filipino</li>
            <li>G: Guamanian</li>
            <li>H: Hispanic/Latin/Mexican</li>
            <li>I: American Indian/Alaskan Native</li>
            <li>J: Japanese</li>
            <li>K: Korean</li>
            <li>L: Laotian</li>
            <li>O: Other</li>
            <li>P: Pacific Islander</li>
            <li>S: Samoan</li>
            <li>U: Hawaiian</li>
            <li>V: Vietnamese</li>
            <li>W: White</li>
            <li>X: Unknown</li>
            <li>Z: Asian Indian</li>
        </ul>
    <li>Premis Cd: Type of structure/vehicle/location where crime took place</li>
    <li>Premis Desc: Defines premis code provided</li>
    <li>Weapon Used Cd: Type of weapon used in crime</li>
    <li>Weapon Used Desc: Defines weapon code provided</li>
    <li>Status: status of case</li>
    <li>Status Desc: Description of status</li>
    <li>Crm Cd 1: Indicates code for primary crime committed </li>
    <li>Crm Cd 2: Additional code for crimes committed</li>
    <li>Crm Cd 3: Additional code for crimes committed</li>
    <li>Crm Cd 4: Additional code for crimes committed</li>
    <li>Location: Street address of crime</li>
    <li>Cross Street: Cross street of rounded address</li>
    <li>Lat: Latitude </li>
    <li>Lon: Longitude</li>
</ul>

In [1]:
import pandas as pd
import numpy as np 

file_path = "/home/francisco/Downloads/Crime_Data_from_2020_to_Present.csv"

df = pd.read_csv(file_path)
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [2]:
df.shape

(847725, 28)

In [3]:
df.dtypes

DR_NO               int64
Date Rptd          object
DATE OCC           object
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float64
LON               float64
dtype: object

In [4]:
df.isnull().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           117642
Vict Age               0
Vict Sex          111909
Vict Descent      111917
Premis Cd             11
Premis Desc          516
Weapon Used Cd    552364
Weapon Desc       552364
Status                 0
Status Desc            0
Crm Cd 1              11
Crm Cd 2          785497
Crm Cd 3          845623
Crm Cd 4          847663
LOCATION               0
Cross Street      712652
LAT                    0
LON                    0
dtype: int64

## Data Cleaning 
<p>Columns are inconsistently formatted, using underscores, dashes, and literal spaces to denote blank space. Some columns are in all caps. 
Several of these columns contain redundant information by providing both the code and the code description. Crm Cd and Crm Cd 1 refer to the exact same data (With the first being of type int and the latter of type float????? My goodness). 
We can create a dictionary for these values and eliminate the description columns.</p>



In [5]:
np.sort(df['Crm Cd'].unique())

array([110, 113, 121, 122, 210, 220, 230, 231, 235, 236, 237, 250, 251,
       310, 320, 330, 331, 341, 343, 345, 347, 349, 350, 351, 352, 353,
       354, 410, 420, 421, 432, 433, 434, 435, 436, 437, 438, 439, 440,
       441, 442, 443, 444, 445, 446, 450, 451, 452, 470, 471, 473, 474,
       475, 480, 485, 487, 510, 520, 522, 622, 623, 624, 625, 626, 627,
       647, 648, 649, 651, 652, 653, 654, 660, 661, 662, 664, 666, 668,
       670, 740, 745, 753, 755, 756, 760, 761, 762, 763, 805, 806, 810,
       812, 813, 814, 815, 820, 821, 822, 830, 840, 845, 850, 860, 865,
       870, 880, 882, 884, 886, 888, 890, 900, 901, 902, 903, 904, 906,
       910, 920, 921, 922, 924, 928, 930, 931, 932, 933, 940, 942, 943,
       944, 946, 948, 949, 950, 951, 954, 956])

In [6]:
crime_codes = df[['Crm Cd', 'Crm Cd Desc']]
crime_codes.head()

,Crm Cd,Crm Cd Desc
0,624,BATTERY - SIMPLE ASSAULT
1,624,BATTERY - SIMPLE ASSAULT
2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE
3,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER)
4,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA..."


In [7]:
crime_codes  = crime_codes.drop_duplicates()
crime_codes.head()

,Crm Cd,Crm Cd Desc
0,624,BATTERY - SIMPLE ASSAULT
2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE
3,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER)
4,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA..."
5,121,"RAPE, FORCIBLE"


In [8]:
crime_code_list  = [code for code in crime_codes['Crm Cd']]
description_list = [desc for desc in crime_codes['Crm Cd Desc']]

crime_code_dictionary = dict(zip(crime_code_list, description_list))

    

In [9]:
crime_code_dictionary

{624: 'BATTERY - SIMPLE ASSAULT',
 845: 'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE',
 745: 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)',
 740: 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
 121: 'RAPE, FORCIBLE',
 442: 'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
 946: 'OTHER MISCELLANEOUS CRIME',
 341: 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD',
 330: 'BURGLARY FROM VEHICLE',
 930: 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
 648: 'ARSON',
 626: 'INTIMATE PARTNER - SIMPLE ASSAULT',
 440: 'THEFT PLAIN - PETTY ($950 & UNDER)',
 354: 'THEFT OF IDENTITY',
 210: 'ROBBERY',
 230: 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 310: 'BURGLARY',
 510: 'VEHICLE - STOLEN',
 420: 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
 761: 'BRANDISH WEAPON',
 236: 'INTIMATE PARTNER - AGGRAVATED ASSAULT',
 662: 'BUNCO, GRAND THEFT',
 350: 'THEFT, PERSON',
 860: 'BATTERY WITH SEXUAL CONTACT',
 480: 'BIKE - STOLEN',
 623: 'BATTERY POLICE (SIMPLE)',
 956: 'LETTERS, LEWD  -  TE

In [10]:
# We can repeat these steps for the other columns that follow the same pattern, in fact...

def create_reference_dictionary(dataframe, column_1, column_2):
    
    temp_df = dataframe[[column_1, column_2]]
    temp_df = temp_df.drop_duplicates()
    temp_df = temp_df.dropna()
    
    column_1_list = [x for x in temp_df[column_1]]
    column_2_list = [x for x in temp_df[column_2]]
    
    return dict(zip(column_1_list, column_2_list))

In [11]:
premis_reference_dictionary = create_reference_dictionary(df, 'Premis Cd', 'Premis Desc')

premis_reference_dictionary = {int(key):value for (key, value) in premis_reference_dictionary.items()}

In [12]:
# Now for Weapons and status

weapon_reference_dictionary = create_reference_dictionary(df, 'Weapon Used Cd', 'Weapon Desc')
status_reference_dictionary = create_reference_dictionary(df, 'Status', 'Status Desc')

weapon_reference_dictionary

{400.0: 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)',
 500.0: 'UNKNOWN WEAPON/OTHER WEAPON',
 306.0: 'ROCK/THROWN OBJECT',
 511.0: 'VERBAL THREAT',
 204.0: 'FOLDING KNIFE',
 302.0: 'BLUNT INSTRUMENT',
 212.0: 'BOTTLE',
 109.0: 'SEMI-AUTOMATIC PISTOL',
 304.0: 'CLUB/BAT',
 218.0: 'OTHER CUTTING INSTRUMENT',
 102.0: 'HAND GUN',
 515.0: 'PHYSICAL PRESENCE',
 307.0: 'VEHICLE',
 216.0: 'SCISSORS',
 308.0: 'STICK',
 215.0: 'MACHETE',
 207.0: 'OTHER KNIFE',
 104.0: 'SHOTGUN',
 200.0: 'KNIFE WITH BLADE 6INCHES OR LESS',
 506.0: 'FIRE',
 214.0: 'ICE PICK',
 221.0: 'GLASS',
 113.0: 'SIMULATED GUN',
 201.0: 'KNIFE WITH BLADE OVER 6 INCHES IN LENGTH',
 504.0: 'DEMAND NOTE',
 501.0: 'BOMB THREAT',
 312.0: 'PIPE/METAL PIPE',
 106.0: 'UNKNOWN FIREARM',
 512.0: 'MACE/PEPPER SPRAY',
 311.0: 'HAMMER',
 208.0: 'RAZOR',
 107.0: 'OTHER FIREARM',
 301.0: 'BELT FLAILING INSTRUMENT/CHAIN',
 223.0: 'UNKNOWN TYPE CUTTING INSTRUMENT',
 219.0: 'SCREWDRIVER',
 205.0: 'KITCHEN KNIFE',
 114.0: 'AIR PISTOL/REVOLV

In [13]:
np.sort(df['Weapon Used Cd'].unique())

array([101., 102., 103., 104., 105., 106., 107., 108., 109., 110., 111.,
       112., 113., 114., 115., 116., 117., 118., 119., 120., 121., 122.,
       123., 124., 125., 200., 201., 202., 203., 204., 205., 206., 207.,
       208., 209., 210., 211., 212., 213., 214., 215., 216., 217., 218.,
       219., 220., 221., 223., 300., 301., 302., 303., 304., 305., 306.,
       307., 308., 309., 310., 311., 312., 400., 500., 501., 502., 503.,
       504., 505., 506., 507., 508., 509., 510., 511., 512., 513., 514.,
       515., 516.,  nan])

In [14]:
# Again, no reason for the dictionary to have floats as keys 

weapon_reference_dictionary = {int(key):value for (key, value) in weapon_reference_dictionary.items()}

In [15]:
weapon_reference_dictionary

{400: 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)',
 500: 'UNKNOWN WEAPON/OTHER WEAPON',
 306: 'ROCK/THROWN OBJECT',
 511: 'VERBAL THREAT',
 204: 'FOLDING KNIFE',
 302: 'BLUNT INSTRUMENT',
 212: 'BOTTLE',
 109: 'SEMI-AUTOMATIC PISTOL',
 304: 'CLUB/BAT',
 218: 'OTHER CUTTING INSTRUMENT',
 102: 'HAND GUN',
 515: 'PHYSICAL PRESENCE',
 307: 'VEHICLE',
 216: 'SCISSORS',
 308: 'STICK',
 215: 'MACHETE',
 207: 'OTHER KNIFE',
 104: 'SHOTGUN',
 200: 'KNIFE WITH BLADE 6INCHES OR LESS',
 506: 'FIRE',
 214: 'ICE PICK',
 221: 'GLASS',
 113: 'SIMULATED GUN',
 201: 'KNIFE WITH BLADE OVER 6 INCHES IN LENGTH',
 504: 'DEMAND NOTE',
 501: 'BOMB THREAT',
 312: 'PIPE/METAL PIPE',
 106: 'UNKNOWN FIREARM',
 512: 'MACE/PEPPER SPRAY',
 311: 'HAMMER',
 208: 'RAZOR',
 107: 'OTHER FIREARM',
 301: 'BELT FLAILING INSTRUMENT/CHAIN',
 223: 'UNKNOWN TYPE CUTTING INSTRUMENT',
 219: 'SCREWDRIVER',
 205: 'KITCHEN KNIFE',
 114: 'AIR PISTOL/REVOLVER/RIFLE/BB GUN',
 303: 'BRASS KNUCKLES',
 101: 'REVOLVER',
 206: 'SWITCH 

Now that we have our reference dictionaries, we no longer need the additional description columns in our dataframe. We can also drop the DR_NO since this is just a record number for the organization.

In [16]:
df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [17]:
columns_to_drop = [
    'DR_NO','Crm Cd Desc', 'Weapon Desc', 
    'Premis Desc', 'Status Desc', 'Crm Cd 1'
]

cleaner_df = df.drop(columns_to_drop, axis = 1)
cleaner_df.shape


(847725, 22)

In [18]:
percent_null_values = (cleaner_df.isnull().sum()) / len(cleaner_df)

with pd.option_context('display.float_format','{:.2%}'.format):
    display(percent_null_values)

Date Rptd         0.00%
DATE OCC          0.00%
TIME OCC          0.00%
AREA              0.00%
AREA NAME         0.00%
Rpt Dist No       0.00%
Part 1-2          0.00%
Crm Cd            0.00%
Mocodes          13.88%
Vict Age          0.00%
Vict Sex         13.20%
Vict Descent     13.20%
Premis Cd         0.00%
Weapon Used Cd   65.16%
Status            0.00%
Crm Cd 2         92.66%
Crm Cd 3         99.75%
Crm Cd 4         99.99%
LOCATION          0.00%
Cross Street     84.07%
LAT               0.00%
LON               0.00%
dtype: float64

There's something wrong with the Date OCC and Time OCC columns...

In [19]:
df[['DATE OCC', 'TIME OCC']].head(15)


,DATE OCC,TIME OCC
0,01/08/2020 12:00:00 AM,2230
1,01/01/2020 12:00:00 AM,330
2,02/13/2020 12:00:00 AM,1200
3,01/01/2020 12:00:00 AM,1730
4,01/01/2020 12:00:00 AM,415
5,01/01/2020 12:00:00 AM,30
6,01/02/2020 12:00:00 AM,1315
7,01/04/2020 12:00:00 AM,40
8,01/04/2020 12:00:00 AM,200
9,05/26/2020 12:00:00 AM,1925


In [20]:
df['DATE OCC'][0][:10]

'01/08/2020'

In [21]:
new_dates = [date[:10] for date in cleaner_df['DATE OCC'].values ]

In [22]:
cleaner_df['date_occurred'] = new_dates

In [23]:
cleaner_df[['DATE OCC','date_occurred']].head(10)

,DATE OCC,date_occurred
0,01/08/2020 12:00:00 AM,01/08/2020
1,01/01/2020 12:00:00 AM,01/01/2020
2,02/13/2020 12:00:00 AM,02/13/2020
3,01/01/2020 12:00:00 AM,01/01/2020
4,01/01/2020 12:00:00 AM,01/01/2020
5,01/01/2020 12:00:00 AM,01/01/2020
6,01/02/2020 12:00:00 AM,01/02/2020
7,01/04/2020 12:00:00 AM,01/04/2020
8,01/04/2020 12:00:00 AM,01/04/2020
9,05/26/2020 12:00:00 AM,05/26/2020


That's much better.
We also have 6 columns dedicated to the location, which is a bit much considering we can derive most of these data by just the Lat and Long coordinates.
We'll drop the original date occ column, along with the 4 columns for area that are NOT lat and long. 

In [24]:
columns_to_drop = [
    'DATE OCC', 'AREA', 'AREA NAME',
    'LOCATION', 'Cross Street'
]

more_cleaner_df = cleaner_df.drop(columns_to_drop, axis = 1)
more_cleaner_df.head(10)

,Date Rptd,TIME OCC,Rpt Dist No,Part 1-2,Crm Cd,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Weapon Used Cd,Status,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON,date_occurred
0,01/08/2020 12:00:00 AM,2230,377,2,624,0444 0913,36,F,B,501.0,400.0,AO,NaN,NaN,NaN,34.0141,-118.2978,01/08/2020
1,01/02/2020 12:00:00 AM,330,163,2,624,0416 1822 1414,25,M,H,102.0,500.0,IC,NaN,NaN,NaN,34.0459,-118.2545,01/01/2020
2,04/14/2020 12:00:00 AM,1200,155,2,845,1501,0,X,X,726.0,NaN,AA,NaN,NaN,NaN,34.0448,-118.2474,02/13/2020
3,01/01/2020 12:00:00 AM,1730,1543,2,745,0329 1402,76,F,W,502.0,NaN,IC,998.0,NaN,NaN,34.1685,-118.4019,01/01/2020
4,01/01/2020 12:00:00 AM,415,1998,2,740,0329,31,X,X,409.0,NaN,IC,NaN,NaN,NaN,34.2198,-118.4468,01/01/2020
5,01/02/2020 12:00:00 AM,30,163,1,121,0413 1822 1262 1415,25,F,H,735.0,500.0,IC,998.0,NaN,NaN,34.0452,-118.2534,01/01/2020
6,01/02/2020 12:00:00 AM,1315,161,1,442,1402 2004 0344 0387,23,M,H,404.0,NaN,IC,998.0,NaN,NaN,34.0483,-118.2631,01/02/2020
7,01/04/2020 12:00:00 AM,40,155,2,946,1402 0392,0,X,X,726.0,NaN,IC,998.0,NaN,NaN,34.0448,-118.2474,01/04/2020
8,01/04/2020 12:00:00 AM,200,101,1,341,1822 0344 1402,23,M,B,502.0,NaN,IC,998.0,NaN,NaN,34.0677,-118.2398,01/04/2020
9,06/19/2020 12:00:00 AM,1925,1708,1,341,1300 0202 0378 0325,0,X,X,203.0,NaN,AO,NaN,NaN,NaN,34.2864,-118.5021,05/26/2020


Much better! The values in Premis Cd, Weapon Used Cd, and the Crm Cd 2/3/4 columns still need to be updated to ints. We can also fill any of the null values with 0, since all of the coded values have three digits. 

In [25]:
percent_null_values = (more_cleaner_df.isnull().sum()) / len(more_cleaner_df)

with pd.option_context('display.float_format','{:.2%}'.format):
    display(percent_null_values)

Date Rptd         0.00%
TIME OCC          0.00%
Rpt Dist No       0.00%
Part 1-2          0.00%
Crm Cd            0.00%
Mocodes          13.88%
Vict Age          0.00%
Vict Sex         13.20%
Vict Descent     13.20%
Premis Cd         0.00%
Weapon Used Cd   65.16%
Status            0.00%
Crm Cd 2         92.66%
Crm Cd 3         99.75%
Crm Cd 4         99.99%
LAT               0.00%
LON               0.00%
date_occurred     0.00%
dtype: float64

In [26]:
more_cleaner_df['Vict Sex'].value_counts()

Vict Sex
M    349347
F    311659
X     74716
H        93
-         1
Name: count, dtype: int64

In [27]:
# H??? That wasn't in the documentation 

mysterious_biology = more_cleaner_df[more_cleaner_df['Vict Sex'] == 'H']
mysterious_biology.head(10)

,Date Rptd,TIME OCC,Rpt Dist No,Part 1-2,Crm Cd,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Weapon Used Cd,Status,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON,date_occurred
1511,01/03/2020 12:00:00 AM,2200,237,2,740,0329,25,H,F,103.0,NaN,IC,NaN,NaN,NaN,34.0613,-118.2638,01/03/2020
1840,01/20/2020 12:00:00 AM,40,238,1,251,1822 0450 1402,35,H,H,502.0,500.0,IC,998.0,NaN,NaN,34.0616,-118.2586,01/20/2020
4589,01/08/2020 12:00:00 AM,1100,566,2,624,1309 0416 0429 0603 0361 0913,27,H,H,502.0,400.0,IC,NaN,NaN,NaN,33.7333,-118.2814,01/07/2020
17720,10/05/2020 12:00:00 AM,200,769,1,230,0913 0906 0416 1218 1805,35,H,H,101.0,312.0,IC,NaN,NaN,NaN,34.0454,-118.3189,10/05/2020
19368,04/14/2020 12:00:00 AM,1300,721,1,330,0344,33,H,F,101.0,NaN,IC,NaN,NaN,NaN,34.0709,-118.3764,04/14/2020
93800,10/30/2020 12:00:00 AM,1155,909,2,354,0377 0100,59,H,F,248.0,NaN,IC,NaN,NaN,NaN,34.2071,-118.4298,10/30/2020
110175,11/29/2020 12:00:00 AM,1530,2088,2,626,0913 2000 1243 0416 0305,25,H,H,502.0,400.0,AO,NaN,NaN,NaN,34.0418,-118.2852,11/29/2020
127579,05/18/2020 12:00:00 AM,1630,1846,2,354,1822 0100 0930,36,H,B,502.0,NaN,IC,NaN,NaN,NaN,33.9332,-118.2485,05/17/2020
135838,05/13/2020 12:00:00 AM,1716,1322,1,440,0344,49,H,F,145.0,NaN,IC,NaN,NaN,NaN,34.0254,-118.2629,05/09/2020
147269,10/04/2020 12:00:00 AM,1530,1555,1,330,0344 1300,45,H,F,101.0,NaN,IC,NaN,NaN,NaN,34.1545,-118.3703,10/02/2020


In [28]:
for crime in mysterious_biology['Crm Cd']:
    print(f'{crime}: {crime_code_dictionary[crime]}')

740: VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)
251: SHOTS FIRED AT INHABITED DWELLING
624: BATTERY - SIMPLE ASSAULT
230: ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT
330: BURGLARY FROM VEHICLE
354: THEFT OF IDENTITY
626: INTIMATE PARTNER - SIMPLE ASSAULT
354: THEFT OF IDENTITY
440: THEFT PLAIN - PETTY ($950 & UNDER)
330: BURGLARY FROM VEHICLE
440: THEFT PLAIN - PETTY ($950 & UNDER)
210: ROBBERY
410: BURGLARY FROM VEHICLE, ATTEMPTED
230: ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT
330: BURGLARY FROM VEHICLE
330: BURGLARY FROM VEHICLE
231: ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER
330: BURGLARY FROM VEHICLE
624: BATTERY - SIMPLE ASSAULT
354: THEFT OF IDENTITY
330: BURGLARY FROM VEHICLE
624: BATTERY - SIMPLE ASSAULT
330: BURGLARY FROM VEHICLE
330: BURGLARY FROM VEHICLE
440: THEFT PLAIN - PETTY ($950 & UNDER)
901: VIOLATION OF RESTRAINING ORDER
331: THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)
354: THEFT OF IDENTITY
230: ASSAULT WITH DEADLY WEAPON, AGGRAVATED A

Considering an H value for victim sex is not listed in any documentation, and how unlikely it would be for an intersex victim to be identified by an H, we'll change these values to unknown (X)

In [29]:
# Taking care of all of the remaining null values 

# Both Vict Sex and Vict Descent have the same number of null values, while not stated explicitly, the documentation suggests these are crimes against property/things
more_cleaner_df['Vict Sex'] = more_cleaner_df['Vict Sex'].str.replace('H','X')
more_cleaner_df['Vict Sex'] = more_cleaner_df['Vict Sex'].str.replace('-','X')
more_cleaner_df['Vict Sex'] = more_cleaner_df['Vict Sex'].fillna('X')
more_cleaner_df['Vict Descent'] = more_cleaner_df['Vict Descent'].fillna('X')

more_cleaner_df['Mocodes'] = more_cleaner_df['Mocodes'].fillna(0)
more_cleaner_df['Weapon Used Cd'] = more_cleaner_df['Weapon Used Cd'].fillna(0)
more_cleaner_df['Crm Cd 2'] = more_cleaner_df['Crm Cd 2'].fillna(0)
more_cleaner_df['Crm Cd 3'] = more_cleaner_df['Crm Cd 3'].fillna(0)
more_cleaner_df['Crm Cd 4'] = more_cleaner_df['Crm Cd 4'].fillna(0)

In [30]:
percent_null_values = (more_cleaner_df.isnull().sum()) / len(more_cleaner_df)

with pd.option_context('display.float_format','{:.2%}'.format):
    display(percent_null_values)

Date Rptd        0.00%
TIME OCC         0.00%
Rpt Dist No      0.00%
Part 1-2         0.00%
Crm Cd           0.00%
Mocodes          0.00%
Vict Age         0.00%
Vict Sex         0.00%
Vict Descent     0.00%
Premis Cd        0.00%
Weapon Used Cd   0.00%
Status           0.00%
Crm Cd 2         0.00%
Crm Cd 3         0.00%
Crm Cd 4         0.00%
LAT              0.00%
LON              0.00%
date_occurred    0.00%
dtype: float64

In [31]:
# Convert the floats to ints 

more_cleaner_df['Premis Cd'] = more_cleaner_df['Premis Cd'].convert_dtypes()


In [32]:
more_cleaner_df['Premis Cd'].unique()

<IntegerArray>
[501, 102, 726, 502, 409, 735, 404, 203, 101, 108,
 ...
 889, 240, 805, 126, 734, 894, 836, 976, 972, 973]
Length: 314, dtype: Int64

In [33]:
# To avoid unnecessary repeats

def convert_and_fill_nulls(series):
    series = series.convert_dtypes()
    series = series.fillna(0)
    return series

In [34]:
more_cleaner_df['Premis Cd'] = more_cleaner_df['Premis Cd'].fillna(0)

more_cleaner_df['Weapon Used Cd'] = convert_and_fill_nulls(more_cleaner_df['Weapon Used Cd'])

more_cleaner_df['Crm Cd 2'] = convert_and_fill_nulls(more_cleaner_df['Crm Cd 2'])

more_cleaner_df['Crm Cd 3'] = convert_and_fill_nulls(more_cleaner_df['Crm Cd 3'])

more_cleaner_df['Crm Cd 4'] = convert_and_fill_nulls(more_cleaner_df['Crm Cd 4'])

In [35]:
more_cleaner_df.dtypes

Date Rptd          object
TIME OCC            int64
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd           Int64
Weapon Used Cd      Int64
Status             object
Crm Cd 2            Int64
Crm Cd 3            Int64
Crm Cd 4            Int64
LAT               float64
LON               float64
date_occurred      object
dtype: object

### We're almost ready for analysis! 
<ul>A few things left to clean up:
    <li>Change the date occured colum to a date format</li>
    <li>Reorder and rename the columns for consistency</li>

</ul>

In [36]:
more_cleaner_df['date_occurred'] = pd.to_datetime(more_cleaner_df['date_occurred'])
more_cleaner_df['date_reported'] = pd.to_datetime(more_cleaner_df['Date Rptd'])

/tmp/ipykernel_8593/493504116.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  more_cleaner_df['date_reported'] = pd.to_datetime(more_cleaner_df['Date Rptd'])


To consolidate the information in date occurred and date reported into valuable data, we'll create a new column indicating the amount of time that passed before the crime was actually reported 

In [37]:

reporting_lag = more_cleaner_df['date_reported'] - more_cleaner_df['date_occurred']
more_cleaner_df['reporting_lag'] = reporting_lag

In [38]:
rename_dictionary = {
    'TIME OCC':'time_occurred',
    'Part 1-2':'part_offense',
    'Rpt Dist No':'reporting_district',
    'Crm Cd':'crime_code',
    'Mocodes':'mo_codes',
    'Vict Age':'victim_age',
    'Vict Sex':'victim_sex',
    'Vict Descent':'victim_descent',
    'Premis Cd':'premises_code',
    'Weapon Used Cd':'weapon_used_code',
    'Status':'status_code',
    'Crm Cd 2':'crime_code_2',
    'Crm Cd 3':'crime_code_3',
    'Crm Cd 4':'crime_code_4',
    'LAT':'latitude',
    'LON':'longitude'
}

In [39]:
more_cleaner_df = more_cleaner_df.rename(columns = rename_dictionary)

more_cleaner_df.columns

Index(['Date Rptd', 'time_occurred', 'reporting_district', 'part_offense',
       'crime_code', 'mo_codes', 'victim_age', 'victim_sex', 'victim_descent',
       'premises_code', 'weapon_used_code', 'status_code', 'crime_code_2',
       'crime_code_3', 'crime_code_4', 'latitude', 'longitude',
       'date_occurred', 'date_reported', 'reporting_lag'],
      dtype='object')

In [40]:

more_cleaner_df = more_cleaner_df.drop(['Date Rptd', 'date_occurred','date_reported'], axis = 1)

The amount of MO Codes per incident varies greatly across the dataset, and may indicate how complete each incident report could be. In this case, we'll change the MO code column to indicate how many different MO Codes were reported with each incident. 

In [41]:
mcs = (more_cleaner_df['mo_codes'])

str_mcs = [str(x) for x in mcs]

new_list = [list(x.split(" ")) for x in str_mcs]

lens = [len(x) for x in new_list]
more_cleaner_df['mo_codes_reported'] = lens

In [42]:
clean_df = more_cleaner_df[['time_occurred', 'part_offense', 'crime_code', 'crime_code_2', 
                            'crime_code_3', 'crime_code_4', 'victim_age', 'victim_sex', 'victim_descent', 
                            'weapon_used_code', 'premises_code', 'status_code', 'reporting_district', 'reporting_lag',
                            'mo_codes_reported', 'latitude', 'longitude']]

clean_df.head()

,time_occurred,part_offense,crime_code,crime_code_2,crime_code_3,crime_code_4,victim_age,victim_sex,victim_descent,weapon_used_code,premises_code,status_code,reporting_district,reporting_lag,mo_codes_reported,latitude,longitude
0,2230,2,624,0,0,0,36,F,B,400,501,AO,377,0 days,2,34.0141,-118.2978
1,330,2,624,0,0,0,25,M,H,500,102,IC,163,1 days,3,34.0459,-118.2545
2,1200,2,845,0,0,0,0,X,X,0,726,AA,155,61 days,1,34.0448,-118.2474
3,1730,2,745,998,0,0,76,F,W,0,502,IC,1543,0 days,2,34.1685,-118.4019
4,415,2,740,0,0,0,31,X,X,0,409,IC,1998,0 days,1,34.2198,-118.4468


In [ ]:
# vs what we started with 
df.head()

We'll also pickle the reference dictionaries for future use 

In [ ]:
import pickle 


with open('crime_code_reference.pickle', 'wb') as file:
    pickle.dump(crime_code_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)


with open('weapon_reference_dictionary.pkl', 'wb') as file:
    pickle.dump(weapon_reference_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    
with open('status_reference_dictionary.pkl', 'wb') as file:
    pickle.dump(status_reference_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    

with open('premis_reference_dictionary.pkl', 'wb') as file:
    pickle.dump(premis_reference_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    

In [43]:
clean_df['crime_code_2'].value_counts()

crime_code_2
0      785497
998     49745
930      3476
860      1198
761       888
        ...  
943         1
972         1
942         1
950         1
438         1
Name: count, Length: 125, dtype: Int64

In [44]:
multi_crime = clean_df.drop(['crime_code_3','crime_code_4'],axis=1)
multi_crime = multi_crime[multi_crime['crime_code_2'] > 0]
multi_crime.info

<bound method DataFrame.info of         time_occurred  part_offense  crime_code  crime_code_2  victim_age  \
3                1730             2         745           998          76   
5                  30             1         121           998          25   
6                1315             1         442           998          23   
7                  40             2         946           998           0   
8                 200             1         341           998          23   
...               ...           ...         ...           ...         ...   
847574           2030             2         624           946          56   
847601           1055             1         230           998          39   
847614           2130             1         121           998          30   
847709           1400             1         210           624          23   
847718            755             1         310           998          54   

       victim_sex victim_descent  weapon_us

In [ ]:
clean_df.to_pickle('./ml_crime_data.pkl')

## Building the ML Model

In [11]:
# Starting with the original cleaned data set 

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

file_path = './ml_crime_data.pkl' 

df = pd.read_pickle(file_path)

df['crime_code'] = [code // 100 for code in df['crime_code']]
df['crime_code_2'] = [code // 100 for code in df['crime_code_2']]
df['weapon_used_code'] = [code // 100 for code in df['weapon_used_code']]
df['premises_code'] = [code // 100 for code in df['premises_code']]

scaler = MinMaxScaler()

df['victim_age'] = scaler.fit_transform(np.array(df['victim_age']).reshape(-1,1))
df['time_occurred'] = scaler.fit_transform(np.array(df['time_occurred']).reshape(-1,1))
df['latitude'] = scaler.fit_transform(np.array(df['latitude']).reshape(-1,1))   
df['longitude'] = scaler.fit_transform(np.array(df['longitude']).reshape(-1,1))
df['reporting_lag'] = scaler.fit_transform(np.array(df['reporting_lag']).reshape(-1,1))


df = df.drop(['mo_codes','status_code','reporting_district'],axis=1)


df['victim_sex'] = df['victim_sex'].astype('str')
df['victim_descent'] = df['victim_descent'].astype('str')


df['victim_sex'] = df['victim_sex'].apply(lambda x: x.replace(x,'0') if x=='M' else x.replace(x,'1')).astype('float')

df['victim_descent'] = df['victim_descent'].apply(lambda x: x.replace(x,'0') if x=='W' else x.replace(x, '1')).astype('float')

In [8]:
# for x in new_list:
#     while len(x) < 10:
#         x.append(0)

# mo_code_arr = [np.array(x) for x in new_list]
# mo_code_arr = np.asarray(mo_code_arr)
# mo_code_arr.shape

In [9]:
# mo_code_df = pd.DataFrame(mo_code_arr, columns=['mo_code_1','mo_code_2','mo_code_3','mo_code_4','mo_code_5','mo_code_6','mo_code_7','mo_code_8','mo_code_9','mo_code_10'],dtype='float')
# df = df.join(mo_code_df)


In [42]:
from sklearn.model_selection import train_test_split

X = np.asarray(multi_crime.drop('victim_sex',axis=1))
y = np.asarray(multi_crime['victim_sex'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [43]:
X_train.shape

(42567, 11)

In [46]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

params = {'C':[0.1,0.3,0.5,0.8,1,3,5],
          'kernel':['linear','poly','rbf','sigmoid'],
          'gamma':['scale','auto']}

clf = SVC(cache_size=10000,verbose=True)
grid = GridSearchCV(clf, params)
grid.fit(X_train,y_train)

print(grid.best_params_)
print(grid.best_score_)


[LibSVM].............................
*..
*..........
*...
*....
*
optimization finished, #iter = 47032
obj = -2609.230319, rho = -0.287148
nSV = 26123, nBSV = 26098
Total nSV = 26123
[LibSVM].........................
*....
*.............
*.......
*.
*
optimization finished, #iter = 48853
obj = -2591.464694, rho = -0.312814
nSV = 25957, nBSV = 25917
Total nSV = 25957
[LibSVM]..........................
*....
*.........
*.........
*......
*....
*.
*
optimization finished, #iter = 56626
obj = -2587.203206, rho = -0.161479
nSV = 25903, nBSV = 25873
Total nSV = 25903
[LibSVM].........................
*.......
*.........
*......
*........
*.
*
optimization finished, #iter = 53635
obj = -2597.897346, rho = -0.245550
nSV = 26006, nBSV = 25981
Total nSV = 26006
[LibSVM]..........................
*......
*......
*
optimization finished, #iter = 37832
obj = -2599.078650, rho = -0.582558
nSV = 26020, nBSV = 25987
Total nSV = 26020
[LibSVM]..................
*.
*..
*
optimization finished, #iter = 

In [47]:

X = np.asarray(multi_crime.drop('victim_descent',axis=1))
y = np.asarray(multi_crime['victim_descent'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

params = {'C':[0.1,0.3,0.5,0.8,1,3,5],
          'kernel':['linear','poly','rbf','sigmoid'],
          'gamma':['scale','auto']}

clf = SVC(cache_size=10000,verbose=True)
grid = GridSearchCV(clf, params)
grid.fit(X_train,y_train)

print(grid.best_params_)
print(grid.best_score_)



[LibSVM].......
*.
*
optimization finished, #iter = 8143
obj = -1493.000546, rho = 1.012202
nSV = 15132, nBSV = 14777
Total nSV = 15132
[LibSVM].......
*.
*
optimization finished, #iter = 8583
obj = -1493.000710, rho = 1.008404
nSV = 15174, nBSV = 14750
Total nSV = 15174
[LibSVM].......
*..
*
optimization finished, #iter = 9124
obj = -1493.201448, rho = 1.005052
nSV = 15264, nBSV = 14709
Total nSV = 15264
[LibSVM].......
*.
*
optimization finished, #iter = 8907
obj = -1493.200670, rho = 1.010352
nSV = 15242, nBSV = 14716
Total nSV = 15242
[LibSVM].......
*.
*
optimization finished, #iter = 8619
obj = -1493.200525, rho = 1.011413
nSV = 15249, nBSV = 14733
Total nSV = 15249
[LibSVM]........
*....
*
optimization finished, #iter = 12315
obj = -1492.997922, rho = 0.999755
nSV = 15703, nBSV = 13999
Total nSV = 15703
[LibSVM]........
*....
*
optimization finished, #iter = 12226
obj = -1492.998410, rho = 0.999805
nSV = 15670, nBSV = 13981
Total nSV = 15670
[LibSVM]........
*....
*
optimization